<a href="https://colab.research.google.com/github/anny11020/Colab/blob/main/X%E5%85%89%E5%88%86%E9%A1%9E_VGG16_%E5%BB%BA%E7%BD%AE%E7%92%B0%E5%A2%83%2B%E8%A8%93%E7%B7%B4%2B%E5%84%B2%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
print("Python version")
print (sys.version)

Python version
3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]


In [ ]:
BATCH_SIZE = 32
LR = 0.001
STEP_SIZE = 5
GAMMA = 0.1
#LR = 1e-1
EPOCH = 100
VAL_SPLIT = 0.25
#NUM_CORES = 8

In [ ]:
SAVE_PATH = '/content/drive/My Drive/Colab Notebooks/Pytorch/Classify/'

In [ ]:
import datetime
today = datetime.date.today()
print(today)

2020-10-19


# 使用TPU
1. https://colab.research.google.com/github/pytorch/xla/blob/master/contrib/colab/getting-started.ipynb#scrollTo=3P6b3uqfzpDI
2. https://colab.research.google.com/github/pytorch/xla/blob/master/contrib/colab/getting-started.ipynb?hl=zh-cn
3. https://github.com/pytorch/xla/tree/master/contrib/colab
4. https://pytorch-lightning.readthedocs.io/en/0.7.1/tpu.html
5. https://zhuanlan.zhihu.com/p/88931693

In [ ]:
"""
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
"""

In [ ]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 133.2MB 89kB/s 
     |████████████████████████████████| 61kB 2.4MB/s 
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


In [ ]:
"""
# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm
"""

# 安裝 pytorch
 * 最新
  - https://pytorch.org/get-started/locally/
  - pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch. org/whl/torch_stable.html
  - pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
 * 1.0.0
  - pip install torch==1.0.0 torchvision==0.2.1
  - pip install Pillow==6.1


In [ ]:
pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
import torch
print(torch.__version__)

1.6.0+cu101


# 資料載入

In [ ]:
import torchvision
from torchvision import transforms, utils, models
import matplotlib.pyplot as plt

* 參考資料
  1. https://www.cnblogs.com/denny402/p/7512516.html
  2. https://blog.csdn.net/TH_NUM/article/details/80877435


In [ ]:
img_data = torchvision.datasets.ImageFolder('/content/drive/My Drive/NEW/NTUST/醫療Dataset/5839_18613_bundle_archive',
                        transform = transforms.Compose([
                        transforms.Resize(256),
                        transforms.CenterCrop(224),
                        #transforms.RandomHorizontalFlip(),
                        transforms.ToTensor()])
                        )
print(img_data.class_to_idx)

{'images_001': 0, 'images_002': 1, 'images_003': 2, 'images_004': 3, 'images_005': 4, 'images_006': 5, 'images_007': 6, 'images_008': 7, 'images_009': 8, 'images_010': 9, 'images_011': 10, 'images_012': 11}


In [ ]:
print(len(img_data))

82120


# 資料處理 1

In [ ]:
"""
data_loader = torch.utils.data.DataLoader(img_data, batch_size=BATCH_SIZE,shuffle=True)
print(len(data_loader))
4606
"""

'\ndata_loader = torch.utils.data.DataLoader(img_data, batch_size=BATCH_SIZE,shuffle=True)\nprint(len(data_loader))\n4606\n'

In [ ]:
"""
def show_batch(imgs):
    grid = utils.make_grid(imgs,nrow=5)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    plt.title('Batch from dataloader')
"""

"\ndef show_batch(imgs):\n    grid = utils.make_grid(imgs,nrow=5)\n    plt.imshow(grid.numpy().transpose((1, 2, 0)))\n    plt.title('Batch from dataloader')\n"

In [ ]:
"""
for i, (batch_x, batch_y) in enumerate(data_loader):
    if i<4:
        print(i, batch_x.size(), batch_y.size())

        show_batch(batch_x)
        plt.axis('off')
        plt.show()
    else:
      break
"""

"\nfor i, (batch_x, batch_y) in enumerate(data_loader):\n    if i<4:\n        print(i, batch_x.size(), batch_y.size())\n\n        show_batch(batch_x)\n        plt.axis('off')\n        plt.show()\n    else:\n      break\n"

In [ ]:
"""
#没有任何转变，所有返回的还是PIL Image对象
print(img_data[0][1]) #第二维度为1 ，表示label
print(img_data[0][0]) #第二维度为0，表示图片数据
"""

'\n#没有任何转变，所有返回的还是PIL Image对象\nprint(img_data[0][1]) #第二维度为1 ，表示label\nprint(img_data[0][0]) #第二维度为0，表示图片数据\n'

# 資料處理 2

* SPLIT DATA -
TRAIN VALID TEST
https://discuss.pytorch.org/t/how-to-split-dataset-into-test-and-validation-sets/33987/5

In [ ]:
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

In [ ]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [ ]:
datasets = train_val_dataset(img_data,VAL_SPLIT)
print(len(datasets['train']))
print(len(datasets['val']))
# The original dataset is available in the Subset class
print(datasets['train'].dataset)

61590
20530
Dataset ImageFolder
    Number of datapoints: 82120
    Root location: /content/drive/My Drive/NEW/NTUST/醫療Dataset/5839_18613_bundle_archive
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
           )


In [ ]:
dataloaders = {x:torch.utils.data.DataLoader(datasets[x],BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train','val']}
x,y = next(iter(dataloaders['train']))
print(x.shape, y.shape)

torch.Size([32, 3, 224, 224]) torch.Size([32])


# VGG MODEL

* VGG訓練
https://hackmd.io/@lido2370/HyLTOlSn4?type=view

In [ ]:
vgg16 = models.vgg16(pretrained=True)
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# Freeze parameters so we don't backprop through them
for param in vgg16.parameters():
    param.requires_grad = False

PYTORCH ON XLA DEVICES
1. http://pytorch.org/xla/release/1.6/index.html
2. https://zhuanlan.zhihu.com/p/
3. https://www.kaggle.com/piantic/pytorch-tpu
4. https://pytorch.org/xla/release/1.6/index.html

In [ ]:

# fine tune the last classifier layer from 1000 to 12 dim(num. of classifier).
# fine tune 最後一層分類數目從 1000 修改成 12。
vgg16.classifier[6] = torch.nn.Linear(4096,12)
vgg16 = vgg16.cuda() #use GPU


'\n# fine tune the last classifier layer from 1000 to 12 dim(num. of classifier).\n# fine tune 最後一層分類數目從 1000 修改成 12。\nvgg16.classifier[6] = torch.nn.Linear(4096,12)\nvgg16 = vgg16.cuda() #use GPU\n'

In [ ]:

# 將 VGG16 model 打印出來
from torchsummary import summary
summary(vgg16.cuda(), (3, 224, 224))


'\n# 將 VGG16 model 打印出來\nfrom torchsummary import summary\nsummary(vgg16.cuda(), (3, 224, 224))\n'

In [ ]:
# fine tune the last classifier layer from 1000 to 12 dim(num. of classifier).
# fine tune 最後一層分類數目從 1000 修改成 12。
vgg16.classifier[6] = torch.nn.Linear(4096,12)

In [ ]:
"""
devices = xm.xla_device()
#devices =(xm.get_xla_supported_devices(max_devices=NUM_CORES) if NUM_CORES != 0 else [])
torch.set_default_tensor_type('torch.FloatTensor')
print("Devices: {}".format(devices))
"""

Devices: xla:1


In [ ]:
#import torch_xla.distributed.data_parallel as dp

In [ ]:
"""
# Scale learning rate to num cores
learning_rate = LR * max(len(devices), 1)
# Pass [] as device_ids to run using the PyTorch/CPU engine.
model = dp.DataParallel(vgg16, device_ids=devices)
"""

'\n# Scale learning rate to num cores\nlearning_rate = LR * max(len(devices), 1)\n# Pass [] as device_ids to run using the PyTorch/CPU engine.\nmodel = dp.DataParallel(vgg16, device_ids=devices)\n'

In [ ]:
"""
# 使用TPU
vgg16 = vgg16.to(devices)
"""

# TRAIN

* TRAIN MODEL
https://zhuanlan.zhihu.com/p/39455807

In [ ]:
model = vgg16

* scheduler
https://zhuanlan.zhihu.com/p/69411064

In [ ]:
# 選擇優化器 & loss function
#optimizer = torch.optim.Adam(vgg16.parameters(), lr=LR)   # optimize all cnn parameters
optimizer = torch.optim.RMSprop(model.parameters(), lr=LR, alpha=0.9)
loss_func = torch.nn.CrossEntropyLoss()   # the target label is not one-hotted
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, STEP_SIZE, gamma=GAMMA, last_epoch=-1)

* 訓練模型


In [ ]:
pip install tensorboardX

In [ ]:
from tensorboardX import SummaryWriter
writer = SummaryWriter()

In [ ]:
from torch.autograd import Variable

#進度條

* 進度條用法
https://kknews.cc/zh-tw/code/2y6jbx9.html
  1. 在使用tqdm顯示進度條的時候，如果代碼中存在print可能會導致輸出多行進度條，此時可以將print語句改為tqdm.write
  https://blog.csdn.net/kdongyi/article/details/101547216
  2. python tqdm模塊不能單行顯示問題
  https://blog.csdn.net/weixin_42138078/article/details/81215207

In [ ]:
#from tqdm import tqdm

* Progressbar
 - https://stackoverrun.com/cn/q/10682996
 - https://blog.csdn.net/dcrmg/article/details/79525167

In [ ]:
#pip install progressbar

In [ ]:
"""
from progressbar import *

widgets = ['Progress: ',Percentage(), ' ', Bar('#'),' ', Timer(),' ', ETA(), ' ', FileTransferSpeed()]
# pbar = ProgressBar(widgets=widgets, maxval=10*total).start()
# pbar.update(10 * i + 1)
# pbar.finish()
"""

"\nfrom progressbar import *\n\nwidgets = ['Progress: ',Percentage(), ' ', Bar('#'),' ', Timer(),' ', ETA(), ' ', FileTransferSpeed()]\n# pbar = ProgressBar(widgets=widgets, maxval=10*total).start()\n# pbar.update(10 * i + 1)\n# pbar.finish()\n"

In [ ]:
# 在使用本方法之前，请先做如下import
from __future__ import division
import math
import sys

def progressbar(cur, total):
    percent = '{:.2%}'.format(cur / total)
    sys.stdout.write('\r')
    sys.stdout.write("[%-50s] %s" % ('=' * int(math.floor(cur * 50 / total)),percent))
    sys.stdout.flush()

#運算

[batch_size, channels, height, width]
* 錯誤
  1. LABEL CUDA
  https://discuss.pytorch.org/t/attributeerror-int-object-has-no-attribute-cuda/30150/2
  2. INPUT SIZE
  https://blog.csdn.net/weixin_38208741/article/details/97894292
  https://stackoverflow.com/questions/57237381/runtimeerror-expected-4-dimensional-input-for-4-dimensional-weight-32-3-3-but
  3. SQUEEZE
  https://blog.csdn.net/xiexu911/article/details/80820028

* 解決
  1. input unsqueeze
  2. label unsqueeze

In [ ]:
CUT = 1

In [ ]:
running_loss = 0.0
running_corrects = 0.0
data_count = 0
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
        global running_loss
        global running_corrects
        global data_count
        # TPU
        model.to(dev)

        for epoch in range(num_epochs):
            print('\n\n----- Epoch {}/{} -----\n'.format(epoch, num_epochs - 1))
            for phase in ['train', 'val']:

                if phase == 'train':
                    scheduler.step()
                    model.train(True)
                else:
                    model.train(False)
                    running_loss = 0.0
                    running_corrects = 0.0

                RUN_TOTAL = int(len(datasets[phase]) / CUT)

                for data in datasets[phase]:
                  progressbar(data_count,RUN_TOTAL)
                  if data_count >= RUN_TOTAL:
                    data_count = 0
                    break

                  inputs, labels = data
                  """
                  # TPU
                  inputs = Variable(inputs.to(dev))
                  inputs = inputs.unsqueeze(0)
                  labels = torch.tensor(labels)
                  labels = labels.unsqueeze(0)
                  labels = Variable(labels.to(dev))
                  """

                  # GPU
                  if torch.cuda.is_available():
                      #inputs = Variable(inputs.cuda())
                      inputs = Variable(inputs.to(dev))
                      inputs = inputs.unsqueeze(0)
                      labels = torch.tensor(labels)
                      labels = labels.unsqueeze(0)
                      #labels = Variable(labels.cuda())
                      labels = Variable(labels.to(dev))
                  else:
                      inputs, labels = Variable(inputs), Variable(labels)


                  optimizer.zero_grad()

                  outputs = model(inputs)
                  #outputs = model(inputs.permute(1,0,2,3))
                  #outputs = model(inputs[None,...])

                  _, preds = torch.max(outputs.data, 1)
                  loss = criterion(outputs, labels)
                  if phase == 'train':
                      loss.backward()
                      optimizer.step()
                      #xm.optimizer_step(optimizer, barrier=True)

                  running_loss += loss.data.item()
                  running_corrects += torch.sum(preds == labels).item()

                  data_count += 1

                epoch_loss = running_loss / RUN_TOTAL
                epoch_acc = running_corrects / RUN_TOTAL

                if phase == 'train':
                    writer.add_scalar('data/trainloss', epoch_loss, epoch)
                    writer.add_scalar('data/trainacc', epoch_acc, epoch)
                else:
                    writer.add_scalar('data/valloss', epoch_loss, epoch)
                    writer.add_scalar('data/valacc', epoch_acc, epoch)

                print('\n{} - Loss: {:.4f} Acc: {:.4f}\n'.format(phase, epoch_loss, epoch_acc))

                save_name = SAVE_PATH+'/CheckPoint_'+str(today)+'_'+str(epoch)+'.pth'
                torch.save({'epoch': epoch,
                      'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      'loss': epoch_loss,}, save_name)

                """
                save_name_output = SAVE_PATH+'/output_'+today+'_'+epoch+'.pth'
                torch.save(output, save_name_output)
                save_name_model = SAVE_PATH+'/model_'+today+'_'+epoch+'.pth'
                torch.save(model, save_name_model)
                """

        writer.export_scalars_to_json(SAVE_PATH+"/all_scalars.json")
        writer.close()
        return model

In [ ]:
#載入先前資料

save_file = SAVE_PATH+"/2020-10-18/CheckPoint_2020-10-18_24.pth"
checkpoint = torch.load(save_file)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print('epoch: ',epoch)
print('loss: ',loss)

epoch:  24
loss:  43.43054459778577


In [ ]:
output = train_model(model,loss_func,optimizer,scheduler,EPOCH)



----- Epoch 0/99 -----

[                                                  ] 0.00%

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[=                                                 ] 2.31%

In [ ]:
torch.save(output, SAVE_PATH+'/model_'+str(today).pth')

# LOAD

CHECK POINT
https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html

In [ ]:
"""
model = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()
"""

MODEL

In [ ]:
#model = torch.load(SAVE_PATH+'./model_20201016.pth')